<a href="https://colab.research.google.com/github/Ankan-Nayak/CodSoft/blob/main/Genre_Classification_Dataset_IMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hijest_genre_classification_dataset_imdb_path = kagglehub.dataset_download('hijest/genre-classification-dataset-imdb')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt",sep=':::',names=['title', 'genre', 'description'],engine="python")
df_test = pd.read_csv("/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt",sep=':::',names=['id', 'title', 'description'],engine="python")

In [ ]:
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
categories= df['genre'].unique()
categories

In [ ]:
values= df['genre'].value_counts()
values

In [ ]:
plt.figure(figsize=(20,10))
colors = [ '#264653']
plt.bar(categories,values,color=colors)
plt.tick_params(axis='x', labelsize=12, rotation=45)
plt.title('Number Of Movies By Gener',fontsize=20)
plt.ylabel('Number of Movies',fontsize=16)
plt.xlabel('Genre',fontsize=16)
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
colors = [ '#264653']
sns.countplot(data= df, y='genre', order= df['genre'].value_counts().index,color= colors[0])
plt.title('Number Of Movies By Gener',fontsize=20)
plt.ylabel('Genre',fontsize=16)
plt.xlabel('Number Of Movies',fontsize=16)
ax = plt.gca()
ax.set_xticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
for p in ax.patches:
    ax.annotate(f'{int(p.get_width())}',
                (p.get_width(), p.get_y() + p.get_height() / 2),
                ha='center', va='center',
                fontsize=12, color='black',
                xytext=(20, 0), textcoords='offset points')
plt.show()

In [ ]:
df.columns

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stemmer = LancasterStemmer()
stop_words = set(stopwords.words('english'))
def text_cleaner(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '',text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'pic.\S+', '', text)
    text = re.sub(r"[^a-zA-Z+']",' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ',text + ' ')
    text = "".join([i for i in text if i not in string.punctuation])
    words = nltk.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    text = " ".join([i for i in words if i not in stopwords and len(i) > 2])
    text = re.sub("\s[\s]+", " ", text).strip()
    return text


In [ ]:
print(nltk.data.path)

In [ ]:
df['Description_cleand'] = df['description'].apply(text_cleaner)
df_test['Desciption_cleaned_test'] = df_test['description'].apply(text_cleaner)

In [ ]:
df['Description_len'] = df['description'].apply(len)
df['Description_cleand_len'] = df['Description_cleand'].apply(len)

In [ ]:
df

In [ ]:
df_test

In [ ]:
plt.figure(figsize=(12, 6))
sns.kdeplot(df['Description_len'], label='Description length',alpha=0.5)
sns.kdeplot(df['Description_cleand_len'], label='Description cleaned length',alpha=0.5)
ax = plt.gca()
#ax.set_xticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#ax.spines['bottom'].set_visible(False)
plt.title('Distribution of Text Lengths')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.legend()



In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X_trian = tfidf_vectorizer.fit_transform(df['Description_cleand'])
X_test = tfidf_vectorizer.transform(df_test['Desciption_cleaned_test'])

In [ ]:
print('shape_train : ', X_trian.shape)
print('shape_test : ', X_test.shape)

In [ ]:
X = X_trian
y = df['genre']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
classifier = MultinomialNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)
print(classification_report(y_val, y_pred))

In [ ]:
X_test_predictions = classifier.predict(X_test)
df_test['Predicted_Genre'] = X_test_predictions

In [ ]:
df_test